In [1]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
import os

In [2]:
import zipfile

data_r = zipfile.ZipFile('train.zip', 'r')
data_r.extractall()

In [3]:
data_r = zipfile.ZipFile('test.zip', 'r')
data_r.extractall()

In [4]:
def download_data(path):
  data = []
  for path_image in sorted(os.listdir(path=path)):
    image = Image.open(path + path_image) #Открываем изображение.
    data.append(np.array(image)) #Загружаем пиксели.
  return data

In [5]:
X_train = download_data(r"train/images/")
Y_train = download_data(r"train/masks/")
X_test = download_data(r"test/images/")
Y_test = download_data(r"test/masks/")


In [6]:
def resize(input_image, input_mask):
   input_image = tf.image.resize(input_image, (128, 128), method="nearest")
   input_mask = tf.image.resize(input_mask, (128, 128), method="nearest")
   return input_image, input_mask

In [35]:
X_train, Y_train = resize(X_train, Y_train)

In [36]:
X_test, Y_test = resize(X_test, Y_test)

In [9]:
X_train[0]

array([[[177, 181, 184],
        [205, 209, 212],
        [219, 223, 226],
        ...,
        [226, 229, 246],
        [230, 233, 250],
        [243, 246, 255]],

       [[217, 221, 224],
        [244, 248, 251],
        [252, 255, 255],
        ...,
        [229, 232, 247],
        [217, 220, 237],
        [208, 211, 228]],

       [[234, 238, 241],
        [252, 255, 255],
        [250, 254, 255],
        ...,
        [240, 242, 255],
        [237, 239, 254],
        [225, 228, 243]],

       ...,

       [[201, 198, 209],
        [223, 217, 229],
        [221, 215, 225],
        ...,
        [241, 234, 224],
        [255, 250, 240],
        [255, 251, 241]],

       [[194, 191, 202],
        [217, 211, 223],
        [223, 217, 227],
        ...,
        [222, 215, 205],
        [230, 223, 213],
        [223, 216, 206]],

       [[212, 209, 220],
        [197, 191, 203],
        [179, 173, 183],
        ...,
        [240, 233, 223],
        [255, 247, 238],
        [254, 245, 236]]

In [10]:
len(Y_train)

7

In [11]:
len(Y_train[0])

644

In [12]:
len(Y_train[0][0])

796

In [13]:
Y_train[0][0][0]

array([132,  41, 246, 255], dtype=uint8)

In [14]:
len(Y_train[0][0][0])

4

In [15]:
palette = {0 : (60, 16, 152), # Building
           1 : (132, 41, 246), # Land
           2 : (110, 193, 228), # Road
           3 : (254, 221, 58), # Vegetation
           4 : (226, 169, 41), # Water
           5 : (155, 155, 155)} # Unlabeled

In [16]:
invert_palette = {v: k for k, v in palette.items()}

In [17]:
# сегментация нейронной сети в RGB изображение
def convert_to_color(arr_2d, palette=palette):
    """ Numeric labels to RGB-color encoding """
    arr_3d = np.zeros((arr_2d.shape[0], arr_2d.shape[1], 3), dtype=np.uint8)

    for c, i in palette.items():
        m = arr_2d == c
        arr_3d[m] = i

    return arr_3d

In [18]:
def convert_from_color(arr_3d, palette=invert_palette):
    """ RGB-color encoding to grayscale labels """
    arr_2d = np.zeros((arr_3d.shape[0], arr_3d.shape[1]), dtype=np.uint8)

    arr_2d = np.zeros((arr_3d.shape[0], arr_3d.shape[1]), dtype=np.int8) # принадлежность каждого пикселя классу
    min_distance = np.zeros((arr_3d.shape[0], arr_3d.shape[1]), dtype=np.float32)+1000 # расстояние до ближайшего класса для пикселей
    for c, i in palette.items():
      distance = np.sum((arr_3d - np.array(c).reshape(1, 1, 3))**2, axis=-1)**(1/2) # ищем расстояние для каждого пикселя до проверяемого класса по евклиду рас-ие
      condition = min_distance > distance # поиск элементов меньше min_distance
      min_distance[condition] = distance[condition] # замена дистанции найденных элементов
      arr_2d[condition] = i # замена класса найденных элементов

    for c, i in palette.items():
      m = np.all(arr_3d == np.array(c).reshape(1, 1, 3), axis=2)
      arr_2d[m] = i

    arr_2d = arr_2d.tolist()
    for i in range(len(arr_2d)):
      for j in range(len(arr_2d[0])):
        label = [0, 0, 0, 0, 0, 0]
        label[arr_2d[i][j]] = 1
        arr_2d[i][j] = label
    arr_2d = np.array(arr_2d)

    return arr_2d

In [37]:
X_train_pred = np.array(X_train).reshape([7, 128, 128, 3])/255
X_test_pred = np.array(X_test).reshape([2, 128, 128, 3])/255
Y_train_pred = []
for i in range(len(Y_train)):
  Y_train_pred.append(convert_from_color(Y_train[i][:, :, :3]))
Y_train_pred = np.array(Y_train_pred)
Y_test_pred = []
for i in range(len(Y_test)):
  Y_test_pred.append(convert_from_color(Y_test[i][:, :, :3]))
Y_test_pred = np.array(Y_test_pred)


In [20]:
import numpy as np
from keras import *
from keras.models import *
from keras.layers import *
from keras.optimizers import *

In [21]:
def double_conv_block(x, n_filters):
   # Conv2D then ReLU activation
   x = layers.Conv2D(n_filters, 3, padding = "same", activation = "relu", kernel_initializer = "he_normal")(x)
   # Conv2D then ReLU activation
   x = layers.Conv2D(n_filters, 3, padding = "same", activation = "relu", kernel_initializer = "he_normal")(x)
   return x

In [22]:
def upsample_block(x, conv_features, n_filters):
   # upsample
   x = layers.Conv2DTranspose(n_filters, 3, 2, padding="same")(x)
   # concatenate
   x = layers.concatenate([x, conv_features])
   # dropout
   x = layers.Dropout(0.3)(x)
   # Conv2D twice with ReLU activation
   x = double_conv_block(x, n_filters)
   return x

In [23]:
def downsample_block(x, n_filters):
   f = double_conv_block(x, n_filters)
   p = layers.MaxPool2D(2)(f)
   p = layers.Dropout(0.3)(p)
   return f, p

In [24]:
def build_unet_model():
   # inputs
   inputs = layers.Input(shape=(128,128,3))
   # encoder: contracting path - downsample
   # 1 - downsample
   f1, p1 = downsample_block(inputs, 64)
   # 2 - downsample
   f2, p2 = downsample_block(p1, 128)
   # 3 - downsample
   f3, p3 = downsample_block(p2, 256)
   # 4 - downsample
   f4, p4 = downsample_block(p3, 512)
   # 5 - bottleneck
   bottleneck = double_conv_block(p4, 1024)
   # decoder: expanding path - upsample
   # 6 - upsample
   u6 = upsample_block(bottleneck, f4, 512)
   # 7 - upsample
   u7 = upsample_block(u6, f3, 256)
   # 8 - upsample
   u8 = upsample_block(u7, f2, 128)
   # 9 - upsample
   u9 = upsample_block(u8, f1, 64)
   # outputs
   outputs = layers.Conv2D(6, 1, padding="same", activation = "softmax")(u9)
   # unet model with Keras Functional API
   unet_model = tf.keras.Model(inputs, outputs, name="U-Net")
   return unet_model

In [25]:
unet_model = build_unet_model()

In [26]:
unet_model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss="categorical_crossentropy",
                  metrics="accuracy")

In [38]:
%%time

history = unet_model.fit(X_train_pred, Y_train_pred,  epochs=10, validation_data = (X_test_pred, Y_test_pred))

Epoch 1/10
1/1 [==============================] - 5s 5s/step - loss: 1.4954 - accuracy: 0.4248 - val_loss: 5.4987 - val_accuracy: 0.6051
Epoch 2/10
1/1 [==============================] - 2s 2s/step - loss: 16.5466 - accuracy: 0.6272 - val_loss: 1.0514 - val_accuracy: 0.6051
Epoch 3/10
1/1 [==============================] - 2s 2s/step - loss: 1.0806 - accuracy: 0.6272 - val_loss: 1.0849 - val_accuracy: 0.6051
Epoch 4/10
1/1 [==============================] - 2s 2s/step - loss: 1.0785 - accuracy: 0.6268 - val_loss: 1.1427 - val_accuracy: 0.6047
Epoch 5/10
1/1 [==============================] - 2s 2s/step - loss: 1.1362 - accuracy: 0.6257 - val_loss: 1.1566 - val_accuracy: 0.6041
Epoch 6/10
1/1 [==============================] - 2s 2s/step - loss: 1.1563 - accuracy: 0.6257 - val_loss: 1.1342 - val_accuracy: 0.6043
Epoch 7/10
1/1 [==============================] - 2s 2s/step - loss: 1.1423 - accuracy: 0.6282 - val_loss: 1.0918 - val_accuracy: 0.6048
Epoch 8/10
1/1 [========================

In [28]:
# def unet_model(image_size, output_classes):

#     #Входной слой
#     input_layer = Input(shape=image_size + (3,))
#     conv_1 = Conv2D(64, 4,
#                                     activation=LeakyReLU(),
#                                     strides=2, padding='same',
#                                     kernel_initializer='glorot_normal',
#                                     use_bias=False)(input_layer)
#     #Сворачиваем
#     conv_1_1 = Conv2D(128, 4,
#                                       activation=LeakyReLU(),
#                                       strides=2,
#                                       padding='same',
#                                       kernel_initializer='glorot_normal',
#                                       use_bias=False)(conv_1)
#     batch_norm_1 = BatchNormalization()(conv_1_1)

#     #2
#     conv_2 = Conv2D(256, 4,
#                                     activation=LeakyReLU(),
#                                     strides=2,
#                                     padding='same',
#                                     kernel_initializer='glorot_normal',
#                                     use_bias=False)(batch_norm_1)
#     batch_norm_2 = BatchNormalization()(conv_2)

#     #3
#     conv_3 = Conv2D(512, 4,
#                                     activation=LeakyReLU(),
#                                     strides=2,
#                                     padding='same',
#                                     kernel_initializer='glorot_normal',
#                                     use_bias=False)(batch_norm_2)
#     batch_norm_3 = BatchNormalization()(conv_3)

#     #4
#     conv_4 = Conv2D(512, 4,
#                                     activation=LeakyReLU(),
#                                     strides=2,
#                                     padding='same',
#                                     kernel_initializer='glorot_normal',
#                                     use_bias=False)(batch_norm_3)
#     batch_norm_4 = BatchNormalization()(conv_4)

#     #5
#     conv_5 = Conv2D(512, 4,
#                                     activation=LeakyReLU(),
#                                     strides=2,
#                                     padding='same',
#                                     kernel_initializer='glorot_normal',
#                                     use_bias=False)(batch_norm_4)
#     batch_norm_5 = BatchNormalization()(conv_5)

#     #6
#     conv_6 = Conv2D(512, 4,
#                                     activation=LeakyReLU(),
#                                     strides=2,
#                                     padding='same',
#                                     kernel_initializer='glorot_normal',
#                                     use_bias=False)(batch_norm_5)
#     ch, cw = get_crop_shape(conv_5,conv_5)
#       #Разворачиваем
#     #1
#     up_1 = Concatenate()([Conv2DTranspose(512, 4, activation='relu', strides=2,
#                                                                           padding='same',
#                                                                           kernel_initializer='glorot_normal',
#                                                                           use_bias=False)(conv_6), conv_5])
#     batch_up_1 = BatchNormalization()(up_1)

#     #Добавим Dropout от переобучения
#     batch_up_1 = Dropout(0.25)(batch_up_1)

#     #2
#     up_2 = Concatenate()([Conv2DTranspose(512, 4, activation='relu', strides=2,
#                                                                           padding='same',
#                                                                           kernel_initializer='glorot_normal',
#                                                                           use_bias=False)(batch_up_1), conv_4])
#     batch_up_2 = BatchNormalization()(up_2)
#     batch_up_2 = Dropout(0.25)(batch_up_2)




#     #3
#     up_3 = Concatenate()([Conv2DTranspose(512, 4, activation='relu', strides=2,
#                                                                           padding='same',
#                                                                           kernel_initializer='glorot_normal',
#                                                                           use_bias=False)(batch_up_2), conv_3])
#     batch_up_3 = BatchNormalization()(up_3)
#     batch_up_3 = Dropout(0.25)(batch_up_3)




#     #4
#     up_4 = Concatenate()([Conv2DTranspose(256, 4, activation='relu', strides=2,
#                                                                           padding='same',
#                                                                           kernel_initializer='glorot_normal',
#                                                                           use_bias=False)(batch_up_3), conv_2])
#     batch_up_4 = BatchNormalization()(up_4)


#     #5
#     up_5 = Concatenate()([Conv2DTranspose(128, 4, activation='relu', strides=2,
#                                                                           padding='same',
#                                                                           kernel_initializer='glorot_normal',
#                                                                           use_bias=False)(batch_up_4), conv_1_1])
#     batch_up_5 = BatchNormalization()(up_5)


#     #6
#     up_6 = Concatenate()([Conv2DTranspose(64, 4, activation='relu', strides=2,
#                                                                           padding='same',
#                                                                           kernel_initializer='glorot_normal',
#                                                                           use_bias=False)(batch_up_5), conv_1])
#     batch_up_6 = BatchNormalization()(up_6)


#     #Выходной слой
#     output_layer = Conv2DTranspose(output_classes, 4, activation='sigmoid', strides=2,
#                                                    padding='same',
#                                                    kernel_initializer='glorot_normal')(batch_up_6)

#     model = Model(inputs=input_layer, outputs=output_layer)
#     return model

In [29]:
from keras.losses import *

In [30]:
import tensorflow as tf

In [40]:
model = unet_model((796,644), 6)

ValueError: Layer "U-Net" expects 1 input(s), but it received 2 input tensors. Inputs received: [<tf.Tensor: shape=(), dtype=int32, numpy=796>, <tf.Tensor: shape=(), dtype=int32, numpy=644>]

In [32]:
#Функция для подсчёта DICE коэффициента
def dice_coef(y_pred, y_true):
    y_pred = tf.unstack(y_pred, axis=3)
    y_true = tf.unstack(y_true, axis=3)
    dice_summ = 0

    for i, (a_y_pred, b_y_true) in enumerate(zip(y_pred, y_true)):
        dice_calculate = (2 * tf.math.reduce_sum(a_y_pred * b_y_true) + 1) /\
         (tf.math.reduce_sum(a_y_pred + b_y_true) + 1)

        dice_summ += dice_calculate
    avg_dice = dice_summ/CLASSES
    return avg_dice

In [33]:
#Функция для подсчета DICE loss
def dice_loss(y_pred, y_true):
    d_loss = 1 - dice_coef(y_pred, y_true)
    return d_loss

In [34]:
# Binary crossentropy + 0.25 * DICE
def dice_bce_loss(y_pred, y_true):
    total_loss = 0.25 * dice_loss(y_pred, y_true) + keras.losses.binary_crossentropy(y_pred, y_true)
    return total_loss